#### -- Exercise 4: Disordering of graphene, and investigating two thermostats-- #
## 0. The thermostats

During this exercise we will test two different thermostats:

- Langevin ensemble

- CSVR, a flavour of velocity rescaling with interesting properties



Then we will use one of those thermostats to "**disorder our graphene**". 

All simulations will be performed with CP2K.
Some very fundamental concept in the structure of the input file are available here:

https://manual.cp2k.org/cp2k-2022_1-branch/index.html






### Visualization functions

In [1]:
def view_structure(structure,myvec=[]):
    t = nv.ASEStructure(structure)
    w = nv.NGLWidget(t, gui=True)
    w.add_unitcell()
    w.add_ball_and_stick()
    w.add_representation('label',label_type='atomindex',color='black')
    w.add_representation('spacefill',selection=myvec,color="blue",radius=0.5)
    return w

def view_trajectory(trajectory,myvec=[]):
    t2 = nv.ASETrajectory(trajectory)
    w2 = nv.NGLWidget(t2, gui=True)
    #w2.add_unitcell()
    w2.add_ball_and_stick()
    w2.add_representation('spacefill',selection=myvec,color="blue",radius=0.5)
    return w2

### Building the system

As last time, we build a graphene sheet. We then rattle a little bit the atoms, and write the coordinates on an ext_xyz file. This has the cell in the second line, which will be nice for our input file.  

In [2]:
import numpy as np
from ase import Atoms
from ase.io import read
from ase.calculators.lammpsrun import LAMMPS
from ase.calculators.cp2k import CP2K
from ase.optimize import BFGS
from ase.build import graphene
from ase.visualize import view
import matplotlib.pyplot as plt
import nglview as nv

# Graphene lattice


graph = graphene(formula='C2',a=2.46,vacuum=1.0,size=(15,15,1))
print (len(graph)," Carbon atoms")



450  Carbon atoms


In [3]:
graph.set_pbc([False,True,False])


view_structure(graph)


NGLWidget()

In [4]:
graph.write("graphene.xyz",format="extxyz")

## First part, running CSVR with different values of the time constant

Create a directory ```CSVR``` and copy there all the input files:

```cp *inp CSVR```

Then, edit them in the terminal to include the appropriate thermostat section: either ```langevin.inc``` or ```csvr.inc```

Run the following input files: 

```start.inp``` (without restart)
```nvt.inp``` (restarting)
```nve.inp```  (restarting)

At the end of each simulation, you can view the trajectory and harvest the data. 
The typical sequence is:

In [ ]:
!cd CSVR ; cp2k.psmp -in start.inp  > start.out

In [ ]:
traj_start = read("CSVR/MD-pos-1.xyz",index=':')

In [ ]:
view_trajectory (traj_start)

In [ ]:
data_therm  = np.loadtxt('CSVR/MD-1.ener')
data_therm  = np.transpose(data_therm)
plt.plot(data_therm[0],data_therm[3], label='Start')
plt.legend( loc='upper right')
plt.xlabel(' time step')
plt.ylabel('T [K]')

In [ ]:
plt.plot(data_therm[0],data_therm[4], label='Start')
plt.legend( loc='upper right')
plt.xlabel(' time step')
plt.ylabel('Pot. Energy (Hartree)')

## Disordering of graphene

In this we will run successive simulations at increasing temperatures, using the command ```sed``` that replaces in a file the occurrence of a string with a number. Then cp2k is run to simulate graphene at that temperature.

In [ ]:
# 
# Set up a chain of input files
#
for mytemp in range (300,3000,500):
    !sed "s/_TEMP_/{mytemp}/g" < CSVR/nvt_ramp.inp > CSVR/nvt_{mytemp}.inp
    print ("Running at T = ",mytemp)
    !cd CSVR ; cp2k.psmp -in nvt_{mytemp}.inp   > nvt_{mytemp}.out


Running at T =  300
Running at T =  800
^C
Running at T =  1300


## Assignment 1 
Run simulations at different values of the CSVR time constant. Which differences do you see? Compute the variance of the temperature along the trajectory for different values of the time constant. 

## Assignment 2
Extract the potential energy average on the production runs at 300 K

## Assignment 3
Discuss the trajectory along the ramp. Store the potential energy at the various temperatures of the ramp and make a plot. (Not mandatory) Repeat the exercise after removing some atoms from the system. Note: you will have to write the ```graphene.xyz``` again. 

## Assignment 4-5-6
Repeat the same exercise for Langevin, creating a separate trajectory ```Langevin``` and tuning the GAMMA value in the input

## Assignment 7 
Do the same exercise (only assignment 1-2) for a system with size 5x5 and report all the differences you see. 